# DiploDatos 2021


### Categorización de publicaciones de productos realizadas en Mercado Libre

### 01 - Análisis y Visualización

#### Condiciones generales que aplican a todos los prácticos:
   - Las notebooks tienen que ser 100% reproducibles, es decir al ejecutar las celdas tal cuál como se entrega la notebook se deben obtener los mismos resultados sin errores.
   - Código legible, haciendo buen uso de las celdas de la notebook y en lo posible seguir estándares de código para *Python* (https://www.python.org/dev/peps/pep-0008/).
   - Utilizar celdas tipo *Markdown* para ir guiando el análisis.
   - Limpiar el output de las celdas antes de entregar el notebook (ir a *Kernel* **-->** *Restart Kernel and Clear All Ouputs*).
   - Incluir conclusiones del análisis que se hizo en la sección "Conclusiones". Tratar de aportar valor en esta sección, ser creativo.

## 1. Consignas

#### Sección A:  Estadísticas básicas & Visualizaciones

Por cada uno de los siguientes puntos realizar un análisis para poder responder el enunciado/pregunta y generar alguna gráfica para visualizar los resultados:

1. ¿Cuántas publicaciones de items hay dentro de cada categoría?
2. Proporción de publicaciones en español y portugués dentro de cada categoría.
3. Proporción de label quality dentro de cada categoría.
4. Relación entre el label quality y el idioma.

#### Sección B: Estadísticas de las publicaciones & Visualizaciones

Por cada uno de los siguientes puntos realizar un análisis para poder responder el enunciado/pregunta y generar alguna gráfica para visualizar los resultados:

1. Cantidad promedio de palabras del título de la publicacion por categoría.
2. Análisis general de *stopwords*, números, caracteres especiales, etc.. Puede ser un recuento promedio por publicación, no es necesario realizar una gráfica en este punto.
3. Palabras más frecuentes dentro de cada categoría (sin incluir *stopwords*, números, caracteres especiales, etc.).

Tener en cuenta librerías como *NLTK* y *spaCy* para el procesamiento de texto.

## 2. Código y Análisis

Importaciones necesarias

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

# Make sure it's 4.14.3
plotly.__version__

Lectura de dataset reducido

In [ ]:
df_dataset = pd.read_csv('DataSet/dataset.csv')

Estudiamos el dataset brevemente antes de comenzar a operar sobre el mismo

In [ ]:
df_dataset.describe()

In [ ]:
print(f'Dimensiones: {df_dataset.shape}')

In [ ]:
print(f'Variables: {list(df_dataset.columns)}')

In [ ]:
print(f'Etiquetas: {list(df_dataset.label_quality.unique())}')

In [ ]:
print(f'Lenguajes: {list(df_dataset.language.unique())}')

In [ ]:
print(f'Categorías: {list(df_dataset.category.unique())}')

Tenemos 646760 productos en nuestro dataframe, con las variables
- `title` la mayoría de los títulos son únicos
- `label_quality` solo se separan en *unreliable* y *reliable*
- `language` solo se separan en *portuguese* y *spanish*
- `category` hay un total de veinte categorías diferentes

## Sección A

#### 1 - Contamos la cantidad de publicaciones por categoría.

In [ ]:
category_count = df_dataset.category\
    .value_counts()\
    .reset_index()\
    .rename(columns={'index': 'category', 'category': 'count'})

category_count

In [ ]:
# Cantidad de publicaciones por categoría
sns.catplot(x='count', y='category', data=category_count, kind='bar');

**Conclusión**

- La categoría `PANTS` es la más común (35973 publicaciones).
- La categoría `WALL_CLOCKS` es la menos común (30600 publicaciones).

Podíamos decir que las categorías están bastante balancedas, teniendo en cuenta que la mayor diferencia entre la cantidad de publicaciones por categoría es cercana a 5000.
Obviamente, esto es consecuencia de haber tomar las 20 categorías más repetidas en el dataset original de **ML**.

#### 2 - Proporción de publicaciones en español y portugués dentro de cada categoría.

In [ ]:
# Tomamos las columnas relevantes
relevant_cols = ['category', 'language']

# Agregamos según el lenguaje
category_language_count = df_dataset\
    .groupby(relevant_cols)\
    .agg(language_count=('language', 'count'))\
    .reset_index()

# Cantidad de publicaciones en español y en portugués dentro de cada categoría
sns.catplot(x='language_count',
            y='category',
            hue ='language',
            data=category_language_count,
            kind='bar');

In [ ]:
# Normalizar nos permite obtener la proporción (en lugar de la cantidad)
category_language_count = pd.crosstab(index=df_dataset['category'],
                                      columns=df_dataset['language'],
                                      normalize='index',
                                      margins=True)

category_language_count

In [ ]:
# Visualizamos la proporción de publicaciones en cada idioma, por categoría
category_language_count.plot.barh(stacked=True,
                                  figsize=(14, 8),
                                  title='Proporción de lenguajes por categoría');

**Conclusión**

- La cantidad de publicaciones en cada idioma es similar.
    - Portugués **50.9%**
    - Español **49.1%**

- Hay **12** categorías con mayor cantidad de publicaciones en portugués.
  Por lo tanto, hay **8** categorías con mayor cantidad de publicaciones en español.

- La categoría con mayor proporción de portugués es `MEMORY_CARDS` con **57.8%**.
- La categoría con mayor proporción de español es `REFRIGERATORS` con **54.8%**.

#### 3 - Proporción de etiquetas dentro de cada categoría.

In [ ]:
# Tomamos las columnas relevantes
relevant_cols = ['category', 'label_quality']

# Agregamos según la etiqueta
category_quality_count = df_dataset\
    .groupby(relevant_cols)\
    .agg(quality_count=('label_quality', 'count'))\
    .reset_index()

# Cantidad de etiquetas dentro de cada categoría
sns.catplot(x='quality_count',
            y='category',
            hue ='label_quality',
            data=category_quality_count,
            kind='bar');

In [ ]:
# Normalizar nos permite obtener la proporción (en lugar de la cantidad)
category_quality_count = pd.crosstab(index=df_dataset['category'],
                                     columns=df_dataset['label_quality'],
                                     normalize='index',
                                     margins=True)

category_quality_count

In [ ]:
# Visualizamos la proporción de etiquetas, por categoría
category_quality_count.plot.barh(stacked=True,
                                 figsize=(14, 8),
                                 title='Proporción de etiquetas por categoría');

**Conclusión**

- La cantidad de publicaciones con cada etiqueta es notoriamente diferente.
    - Confiable **14.7%**
    - No Confiable **85.3%**

- Ninguna categoría tiene una mayor cantidad de publicaciones verificadas, que no verificadas.

- La categoría con mayor proporción de publicaciones verificadas es `PANTS` con **22.3%**.
- La categoría con mayor proporción de publicaciones no verificadas es `WINES` con **97.1%**.

#### 4 - Relación entre la etiqueta y el lenguaje.

In [ ]:
# Tomamos las columnas relevantes
relevant_cols = ['category', 'language', 'label_quality']

# Agregamos según la categoría
category_language_quality_count = df_dataset\
    .groupby(relevant_cols)\
    .agg(language_quality_count=('category', 'count'))\
    .reset_index()

# Cantidad de lenguajes / etiquetas dentro de cada categoría
sns.catplot(x='language_quality_count',
            y='category',
            hue='label_quality',
            col='language',
            data=category_language_quality_count,
            kind='bar');

In [ ]:
# Normalizar nos permite obtener la proporción (en lugar de la cantidad)
language_quality_count = pd.crosstab(index=df_dataset['language'],
                                     columns=df_dataset['label_quality'],
                                     normalize=True,
                                     margins=True)

language_quality_count

In [ ]:
# Visualizamos la proporción de etiquetas y de lenguajes
plot_df = language_quality_count.drop(index='All').drop(columns='All')

plot_df.plot.barh(stacked=True,
                  figsize=(14, 8),
                  title='Proporción de etiquetas y de lenguajes');

**Conclusión**

Si se toma un publicación cualquiera del dataset, lo más probable es obtener una publicación en portugués no verificada (**43.1%**), mientras que lo menos probable es obtener una publicación en español verificada (**6.9%**).

## Sección B

In [ ]:
# Copiamos el dataset original
df_nlp = df_dataset.copy()

Necesitamos modelos específicos para cada uno de los lenguajes.

- **Español** `es_core_news_sm`
- **Portugués** `pt_core_news_sm`

In [ ]:
import spacy

"""
Una optimización posible sería cargar solamente los componentes
que vamos a utilizar, es decir, el tokenizer y el tagger.
"""

# Modelo para procesar Español
nlp_es = spacy.load('es_core_news_sm')
# Modelo para procesar Portugués
nlp_pt = spacy.load('pt_core_news_sm')

In [ ]:
nlp_lang = {'spanish': nlp_es, 'portuguese': nlp_pt}

def tokenizer(row):
    """
    Dada una fila del dataset, aplica el procesamiento al título:
    - Tokenizer: separa en tokens
    - Tagger: agrega información sintáctica/semántica
    Es importante procesar de acuerdo al lenguaje de la publicación.
    """
    nlp = nlp_lang[row.language]
    # Por cuestiones de eficiencia, solo aplicamos estas etapas.
    return nlp.tagger(nlp.tokenizer(row.title))

In [ ]:
# Queremos algo manejable para realizar el análisis...
df_nlp = df_nlp.sample(50000, random_state=123)

In [ ]:
df_nlp['tokens'] = df_nlp.apply(tokenizer, axis=1)

df_nlp.sample(5)

#### 1 - Cantidad promedio de palabras en el título de la publicación por categoría.

In [ ]:
# Ejercicio 1: Contar la cantidad de tokens.
df_ammount_tokens = df_nlp.copy()

df_ammount_tokens['ammount_tokens'] = df_ammount_tokens.tokens.apply(lambda tokens: len(tokens))

df_ammount_tokens.sample(5)

In [ ]:
relevant_cols = ['category', 'ammount_tokens']

df_ammount_tokens = df_ammount_tokens[relevant_cols]\
    .groupby('category')\
    .agg(ammount_tokens_mean=('ammount_tokens', 'mean'))\
    .reset_index()

df_ammount_tokens

In [ ]:
# Promedio de la cantidad de tokens por categoría
sns.catplot(x='ammount_tokens_mean',
            y='category',
            data=df_ammount_tokens,
            kind='bar');

**Conclusión**

La categoría con mayor cantidad promedio de tokens en su título es `MEMORY_CARDS` con **10.74**, seguida por `KITCHEN_SINKS` con **9.26**.

La categoría con menor cantidad promedio de tokens en su título es `PUREBRED_DOGS` con **5.98**, seguida por `REFRIGERATORS` con **6.91**.

El resto de las categorías poseen, en promedio, entre 7 a 8 palabras en su título.

#### 2 - Análisis general de *stopwords*, números, caracteres especiales, etc..

**Stop Word**

En computación, **stop words** son palabras que serán filtradas durante el *procesamiento del lenguaje natural* de los datos.
Normalmente refieren a las palabras más comúnes de un lenguaje, donde no existe una lista absoluta definitiva que contenga a todas las **stop words** de un lenguaje.

In [ ]:
def other_POS(token):
    """
    Determina si el token corresponde a alguna de las POS:
    - ENUM: Números
    - PUNCT: Puntuaciones
    - SYM: Símbolos
    - SPACE: Espacios
    """
    return token.pos_ in ['ENUM', 'PUNCT', 'SYM', 'SPACE']

In [ ]:
# Ejercicio 2: Contar la cantidad de stop words y otras POS.
df_ammount_stop_words = df_nlp.copy()

count_stop_words = lambda tokens: sum(map(lambda t: t.is_stop, tokens))
count_others_POS = lambda tokens: sum(map(lambda t: other_POS(t), tokens))

df_ammount_stop_words['ammount_stop_words'] = df_ammount_stop_words.tokens.apply(count_stop_words)
df_ammount_stop_words['ammount_others_POS'] = df_ammount_stop_words.tokens.apply(count_others_POS)

df_ammount_stop_words.sample(5)

In [ ]:
relevant_cols = ['category', 'ammount_stop_words', 'ammount_others_POS']

df_ammount_stop_words = df_ammount_stop_words[relevant_cols]\
    .groupby('category')\
    .agg(ammount_stop_words_mean=('ammount_stop_words', 'mean'),
         ammount_others_POS_mean=('ammount_others_POS', 'mean')
        )\
    .reset_index()

df_ammount_stop_words

In [ ]:
# Promedio de cantidad de stop words (y otras POS) por categoría
fig, axes = plt.subplots(2, 1, figsize=(14, 14), sharex=True, sharey=True)

# Stop Words
sns.barplot(x='ammount_stop_words_mean',
            y='category',
            data=df_ammount_stop_words,
            ax=axes[0]);

# Otras POS
sns.barplot(x='ammount_others_POS_mean',
            y='category',
            data=df_ammount_stop_words,
            ax=axes[1]);

**Conclusión**

Es evidente que es más común la ocurrencia de *stop words* en el título de alguna publicación, que el uso de otras *POS* (**Part Of Speech**), al menos en la mayoría de las categorías.

La categoría con mayor cantidad promedio de *stop words* en su título es `WALL_CLOCKS` con **1.27**.

La categoría con menor cantidad promedio de *stop words* en su título es `MOTORCYCLE_JACKETS` con **0.30**.

La categoría con mayor cantidad promedio de otras *POS* en su título es `WINES` con **0.74**.

La categoría con menor cantidad promedio de otras *POS* en su título es `KITCHEN_SINKS` con **0.36**.

#### 3 - Palabras más frecuentes dentro de cada categoría.

In [ ]:
def valid_word(token):
    """
    Determina si un token corresponde a una palabra válida.
    Es decir, no es alguna de las otras POS (ENUM, PUNCT,
    SYM, SPACE), ni tampoco una stop word.
    """
    return not (token.is_stop or other_POS(token))

In [ ]:
# Ejercicio 3: Encontrar las palabras más repetidas.
from collections import Counter

df_ammount_words = df_nlp.copy()

count_words = lambda tokens: Counter([t.text for t in tokens if valid_word(t)])

df_ammount_words['ammount_words'] = df_ammount_words.tokens.apply(count_words)

df_ammount_words.sample(5)

In [ ]:
def agg_counter_sum(series):
    """
    Cada publicación tendrá un conteo de las palabras utilizadas en su título.
    Agrupando por categoría, sumamos los contadores de cada publicación.
    """
    return sum(series, Counter())

In [ ]:
relevant_cols = ['category', 'ammount_words']

df_ammount_words = df_ammount_words[relevant_cols]\
    .groupby('category')\
    .agg(ammount_words_counter=('ammount_words', agg_counter_sum))\
    .reset_index()

df_ammount_words

In [ ]:
# Nos quedamos con las 'top_words' de cada categoría.
top_words = 3

top_words_df = df_ammount_words.copy()

top_words_df['ammount_words_counter'] = top_words_df['ammount_words_counter']\
    .apply(lambda c: c.most_common(top_words))

top_words_df

In [ ]:
# Para un gráfico de barras, deberíamos acomodar nuestro df
list_df = []
for index, row in top_words_df.iterrows():
    for word, count in row.ammount_words_counter:
        list_df.append((row.category, word, count))

plot_df = pd.DataFrame(list_df, columns=['category', 'word', 'count'])

In [ ]:
px.bar(plot_df,
       x='word',
       y='count',
       color='category',
       width=1500,
       height=600,
       barmode='group')

**Conclusión**

La mayoría de las palabras más comunes resultan lógicas al ver los resultados obtenidos.

La palabra más repetida es **Teclado** con 1853, en la categoría `MUSICAL_KEYBOARDS`.

Hay curiosidades como...

- La palabra **Maquina** es top tanto para la categoría `HAIR_CLIPPERS` con 692, como en `SEWING_MACHING` con 925.

- Palabras top para referirse al mismo elemento, en cada uno de los idiomas.
  Por ejemplo, en `REFRIGERATORS` tenemos **Heladera** con 1266 y **Geladeira** con 572.

- Los errores de ortografía también son relevantes.
  Por ejemplo, en `MATTRESSES` tenemos **Colchon** con 722 y **Colchón** con 498.

- Una palabra puede aparecer en singular o en plural.
  Por ejemplo, en `PUREBRED_DOGS` tenemos **Filhotes** con 437 y **Filhote** con 304.

## 3. Conclusiones

Las conclusiones particulares de cada ejercicio se fueron anotando al final de cada uno.

Como conclusión general, podemos decir que estamos frente a un problema de clasificación complejo, principalmente por la naturaleza del *lenguaje natural*.
Además se suma la dificultad de trabajar con dos idiomas al mismo tiempo, español y portugués.

El análisis nos aportó una percepción más completa del dataset, y nos ayudó a comenzar a incorporar algunos conceptos básicos de **NLP**.
En particular, la librería **spaCy** para *Python*.